In [1]:
"""
We want an interctive finance chart for monthyl costs, to see affordability based on factors

Costs:
* Utilities
* Insurance
* Property Taxes

Based on factors
* Sale price
* part that is land cost
* Number of roms


For each factor, have line chart, of value vs monthly cost
"""

import pandas as pd
import altair as alt

Start with monthly cost vs proprety cost

In [68]:
property_cost = alt.sequence(
    100 * 1000,
    300 * 1000,
    1000,
    as_='property_cost'
)
interest_rate = alt.sequence(
    0,
    0.10,
    0.01,
    as_='interest_rate'
)

insurance_slider = alt.binding_range(min=300, max=800, step=100)

select_insurance = alt.selection_single(
    name="insurance",
    fields=['insurance'],
    bind=insurance_slider,
    init={'insurance': 500}
)


nearest_property_cost = alt.selection(
    type='single',
    on='mouseover',
    nearest=True,
    fields=['property_cost'],
    init={'property_cost': 270 * 1000}
)
nearest_interest_rate = alt.selection(
    type='single',
    on='mouseover',
    nearest=True,
    fields=['interest_rate'],
    init={'interest_rate': 0.03}
)



property_cost_line = alt.Chart(property_cost).transform_calculate(
    monthly_cost=(alt.datum.property_cost * nearest_interest_rate.interest_rate) / 12  + select_insurance.insurance /12
).mark_line().encode(
    alt.X('property_cost:Q', axis=alt.Axis(format='$.2s')),
    alt.Y('monthly_cost:Q', axis=alt.Axis(format='$.2s')),
).add_selection(
    select_insurance
)

interest_rate_line = alt.Chart(interest_rate).transform_calculate(
    monthly_cost=(nearest_property_cost.property_cost * alt.datum.interest_rate) / 12  + select_insurance.insurance / 12
).mark_line().encode(
    alt.X('interest_rate:Q', axis=alt.Axis(format='%')),
    alt.Y('monthly_cost:Q', axis=alt.Axis(format='$.2s')),
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
property_cost_selectors = alt.Chart(property_cost).mark_point().encode(
    x='property_cost:Q',
    opacity=alt.value(0),
).add_selection(
    nearest_property_cost
)

interest_rate_selectors = alt.Chart(interest_rate).mark_point().encode(
    x='interest_rate:Q',
    opacity=alt.value(0),
).add_selection(
    nearest_interest_rate
)



# Draw a rule at the location of the selection
property_cost_rules = alt.Chart(property_cost).mark_rule(color='gray').encode(
    x='property_cost:Q',
).transform_filter(
    nearest_property_cost
)
interest_rate_rules = alt.Chart(interest_rate).mark_rule(color='gray').encode(
    x='interest_rate:Q',
).transform_filter(
    nearest_interest_rate
)

source = alt.Chart(
    alt.InlineData([
        {"category": "insurance"},
        {"category": "investment"}
    ])
).transform_calculate(
    cost=alt.expr.if_(
        alt.datum.category == 'insurance',
        select_insurance.insurance / 12,
        alt.expr.if_(
            alt.datum.category == 'investment',
            nearest_interest_rate.interest_rate * nearest_property_cost.property_cost / 12,
            None
        )
    )
)


base = source.encode(
    theta=alt.Theta("cost:Q", stack=True),
    tooltip=['category:N', 'cost:Q']
)

pie = base.mark_arc(
    innerRadius=20,
    outerRadius=120
).encode(
    color=alt.Color(
        "category:N",
        legend=alt.Legend(
            orient='left'
        )
    ),
)
text = base.mark_text(
    radius=140,
    size=10
).encode(text="cost:Q")


sum_text = source.mark_text(radius=0, size=20).encode(
    alt.Text("cost:Q", aggregate='sum')
)




chart = alt.vconcat(
    pie + text + sum_text,
    alt.concat(
        alt.layer(property_cost_line, property_cost_selectors, property_cost_rules),
        alt.layer(interest_rate_line, interest_rate_selectors, interest_rate_rules),
        columns=2
    ).resolve_scale(
        y='shared'
    )
)
chart.save('index.html')
chart

alt.VConcatChart(...)